In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep


options= Options()
options.headless = True
driver = webdriver.Chrome(options = options)
url= "https://www.customs.go.jp/news/index.htm"

driver.get(url)
sleep(1)

soup = BeautifulSoup(driver.page_source, 'lxml')
soup = soup.find("div", attrs={"class":"pastList category"}).find('h3')

In [ ]:
dates = []
titles = []
dt = soup.get_text()
idx = dt.find("日")

date = dt[:idx+1]
dates.append(date)
title = dt[idx + 2:]
titles.append(title)

In [ ]:
#가장 최근 뉴스 태그들 가져오기
tags=[]
tag = soup.select("img")
for i in range(len(tag)):
    if len(tag[i]['alt']) > 0:
        tags.append(tag[i]['alt'])
print(tags)

In [ ]:
#가장 최근 뉴스의 제목과 링크 가져오기
topics=[]
links=[]
data = soup.find("a")

topic = data.get_text()
link = "https://www.customs.go.jp"+data["href"]

topics.append(topic)
links.append(link)

print(topics)
print(links)

In [ ]:
#가장 최근 뉴스 날짜, 태그, 제목, 링크 DF형태로 출력
import pandas as pd

df = pd.DataFrame()
df['date'] = dates
df['tag'] = tags[0]
df['topic'] = topics
df['link'] = links
df

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep


options= Options()
options.headless = True
driver = webdriver.Chrome(options = options)
url= "https://www.customs.go.jp/news/index.htm"

driver.get(url)
sleep(1)

soup = BeautifulSoup(driver.page_source, 'lxml')
rows = soup.find("div", attrs={"class":"pastList category"}).find_all('h3')
soup = soup.find("div", attrs={"class":"pastList category"})

In [2]:
#모든 date list에 추가
dates = []
titles = []
for row in rows: 
    A = row.find_all("a")
    for a in A:
        dt = a.get_text()
        idx = dt.find("日")
        date = dt[:idx+1]
        dates.append(date)
        title = dt[idx + 2:]
        titles.append(title)
print(len(dates))
print(len(titles))

209
209


In [3]:
#모든 tag list에 추가
tags = []
tag = soup.find_all("img")
t=['統計', '手続', '採用', '法令', 'その他', '報道資料']
#   통계,  수속(절차), 채용,  법령, 기타,  보도자료

for i in range(len(tag)):
    if tag[i]['alt'] in t:
        tags.append(tag[i]['alt'])
print(len(tags))

209


In [32]:
#모든 title, link list에 추가
titles = []
links = []

datas = soup.find_all("a")

for data in range(len(datas)):
    titles.append(datas[data].get_text())
    b_link = datas[data]['href']
    if b_link[:2] == "..":
        b_link = b_link[2:]
    elif b_link[0] == ".":
        b_link = b_link[1:]
        if b_link[:5] == "/news":
            b_link = "/news" + b_link
    link = "https://www.customs.go.jp" + b_link
    links.append(link)
print(len(titles))
print(len(links))

209
209


In [33]:
import pandas as pd

df = pd.DataFrame()
df['date'] = dates
df['tag'] = tags
df['title'] = titles
df['link'] = links
df

,date,tag,title,link
0,,統計,貿易統計（令和4年7月上旬分速報）,https://www.customs.go.jp/toukei/latest/index.htm
1,税関における新型コロナウイルス感染症陽性者の判明について(令和4年7月27日,その他,税関における新型コロナウイルス感染症陽性者の判明について(令和4年7月27日更新),https://www.customs.go.jp/news/news/2022covid-...
2,価格の換算に用いる外国為替相場（令和4年7月31日,手続,価格の換算に用いる外国為替相場（令和4年7月31日～令和4年8月6日）,https://www.customs.go.jp/tetsuzuki/kawase/ind...
3,,その他,クレジットカードを利用した関税等の納付のページを更新しました。,https://www.customs.go.jp/kaigairyoko/creca.html
4,,その他,スマートフォン決済アプリを利用した関税等の納付のページを更新しました。,https://www.customs.go.jp/kaigairyoko/smapay.htm
...,...,...,...,...
204,,その他,沖縄地区税関における新型コロナウイルス感染症陽性者の判明について,https://www.customs.go.jp/news/news/covid-19_2...
205,,その他,沖縄地区税関における新型コロナウイルス感染症陽性者の判明について,https://www.customs.go.jp/news/news/covid-19_2...
206,,その他,沖縄地区税関における新型コロナウイルス感染症陽性者の判明について,https://www.customs.go.jp/news/news/covid-19_2...
207,実行関税率表（2022年1月1日,手続,実行関税率表（2022年1月1日版）,https://www.customs.go.jp/tariff/2022_01_01/in...


In [35]:
df.to_csv('jc_new.csv', mode='w', encoding='utf-8-sig')